In [ ]:
# import argparse
import configparser
import dateutil
import json
import logging
import os
import sys

import pandas as pd

logging.basicConfig()
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

# parser = argparse.ArgumentParser()
# parser.add_argument(
#     "-c", "--config", type=str, default="settings.conf", help="Path to config file."
# )
# args = parser.parse_args()

# config_file = args.config

config_file = 'settings.conf'

config = configparser.RawConfigParser()
config.read(config_file)

df_header = [
    ('timestamp', 'timestamp'),
    ('user_name', 'author.name'),
    ('user_id', 'author.id'),
    ('bot', 'author.isBot'),
    ('channel', 'channel.name'),
    ('message', 'content'),
    ('message_id', 'id')
]

In [2]:
sys.path.append(os.path.abspath(os.path.join('src/')))

from process_data import ProcessData

In [3]:
proc = ProcessData(config_path=config_file)

data = proc.create_dataframe(query={})

print(proc.data)

NameError: name 'create_dataframe' is not defined

In [ ]:
from google.cloud import language_v1beta2 as lang

client = lang.LanguageServiceClient()

document = {
    'content': '',
    'type_': lang.types.Document.Type.PLAIN_TEXT,
}

encoding_type = lang.EncodingType.UTF32

sentiment_column = []
for idx, msg in data.iterrows():
    document['content'] = msg['message']
    print(document)

    response = client.analyze_entity_sentiment(
        request={
            'document': document,
            'encoding_type': encoding_type
        }
    )
    print(response)
    sentiment_column.append(response)

data['sentiment'] = sentiment_column

print(data)

In [ ]:
users = []
for name in data['user_name'].unique():
    if '#' not in name:
        users.append(name)

print(users)

In [ ]:
for user in users:
    user_data = data[data['user_name'] == user]
    print(f"{user} - {user_data['message'].count()}")

user_summary = pd.DataFrame()